# MTA Trip Analysis

In [107]:
# Lib Import
import pandas as pd
import numpy as np

# This line is needed to display plots inline in Jupyter Notebook
%matplotlib inline

# Required for basic python plotting functionality
import matplotlib.pyplot as plt

# Required for formatting dates later in the case
import datetime
import matplotlib.dates as mdates

In [108]:
mtadf = pd.read_csv('Assets/mta_trips.csv')
geography = pd.read_csv('Assets/geographic.csv')
demographics = pd.read_csv('Assets/demographics.csv')
weather = pd.read_csv('Assets/weather.csv')
zones = pd.read_csv('Assets/zones.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [109]:
#str(mtadf['datetime'][0])[6:10]+'-'+str(mtadf['datetime'][0])[0:2]+'-'+str(mtadf['datetime'][0])[3:5]+' '+str(mtadf['datetime'][0])[11:16]
mtadf['datetime'] = mtadf['datetime'].apply( lambda row: datetime.datetime.strptime(str(row)[6:10]+'-'+str(row)[0:2]+'-'+str(row)[3:5]+' '+str(row)[11:16], "%Y-%m-%d %H:%M"))

In [110]:
stations = mtadf[['station', 'longitude', 'latitude']]

In [111]:
stations = stations.groupby('station')['latitude','longitude'].mean().reset_index()
stations.head()

,station,latitude,longitude
0,1 AVE,40.730953,-73.981628
1,103 ST,40.796092,-73.961454
2,104 ST,40.695178,-73.844330
3,110 ST,40.795020,-73.944250
4,110 ST-CATHEDRL,40.800605,-73.958158


In [112]:
import folium  #needed for interactive map
from folium.plugins import HeatMap

folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="OpenStreetMap")

for i in range(0,stations.shape[0]):
    marker = folium.CircleMarker(location=[stations["latitude"][i],stations["longitude"][i]],radius=5,color="red",fill=True)
    marker.add_to(folium_map)
folium_map

In [113]:
passengers = mtadf.groupby('station')['latitude','longitude','new_entries', 'new_exits'].mean().reset_index()
passengers.head()

,station,latitude,longitude,new_entries,new_exits
0,1 AVE,40.730953,-73.981628,345.664767,342.886687
1,103 ST,40.796092,-73.961454,337.613355,210.227419
2,104 ST,40.695178,-73.844330,101.020047,72.228066
3,110 ST,40.795020,-73.944250,312.563855,244.552429
4,110 ST-CATHEDRL,40.800605,-73.958158,296.740345,151.391458


In [114]:
max_amount = float(passengers['new_entries'].max())
folium_hmap = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="OpenStreetMap")

hm_wide = HeatMap( list(zip(passengers['latitude'], passengers['longitude'], passengers['new_entries'])),
                   min_opacity=0.9,
                   max_val=max_amount,
                   radius=8, blur=10, 
                   max_zoom=15, 
                 )

folium_hmap.add_child(hm_wide)

In [106]:
max_amount = float(passengers['new_exits'].max())
folium_hmap = folium.Map(location=[40.738, -73.98],
                        zoom_start=11,
                        tiles="OpenStreetMap")

hm_wide = HeatMap( list(zip(passengers['latitude'], passengers['longitude'], passengers['new_exits'])),
                   min_opacity=0.9,
                   max_val=max_amount,
                   radius=8, blur=10, 
                   max_zoom=15, 
                 )

folium_hmap.add_child(hm_wide)

In [137]:
Flow_in_one_day = mtadf[(mtadf['datetime'] > '2014-03-29') & (mtadf['datetime'] < '2014-03-30')]
Flow_in_one_day = Flow_in_one_day.groupby(['station', 'datetime'])['new_entries', 'new_exits'].sum().reset_index()

In [145]:
Flow_in_one_day.head(15)

,station,datetime,new_entries,new_exits
0,1 AVE,2014-03-29 04:00:00,2295,1565
1,1 AVE,2014-03-29 08:00:00,776,1017
2,1 AVE,2014-03-29 12:00:00,2569,2919
3,1 AVE,2014-03-29 16:00:00,4954,5120
4,1 AVE,2014-03-29 20:00:00,4752,6632
5,103 ST,2014-03-29 05:00:00,406,647
6,103 ST,2014-03-29 09:00:00,2271,1272
7,103 ST,2014-03-29 13:00:00,5942,2326
8,103 ST,2014-03-29 17:00:00,5863,3416
9,103 ST,2014-03-29 21:00:00,4827,3328


In [139]:
stations_list = list(set(Flow_in_one_day['station']))

In [140]:
Ss=Flow_in_one_day[Flow_in_one_day['station']==stations_list[0]]

In [152]:
mtadf['audit_type'].value_counts()

REGULAR       7422940
DOOR OPEN       66192
RECOVR AUD      36418
DOOR CLOSE      21543
LOGON            3821
LGF-MAN          3276
REGULAR.1           1
REGULAR.3           1
REGULAR.6           1
REGULAR.2           1
REGULAR.4           1
REGULAR.7           1
REGULAR.5           1
Name: audit_type, dtype: int64